In [1]:
import json
import random
import pickle
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import SGD

In [2]:
# Load training data
lemmatizer = WordNetLemmatizer()
data = open('Data/responses.json').read()
intents = json.loads(data)
words, classes, documents = [], [], []
ignore_words = ['?', '!']



In [3]:
# Data Preprocessing
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        w = word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent["tag"]))
        if intent['tag'] not in classes:
            classes.append(intent["tag"])

words = sorted(list(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words])))
classes = sorted(list(set(classes)))


In [4]:
# Print data information
print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

29 documents
7 classes ['angry', 'anxious', 'goodbye', 'greeting', 'happy', 'sad', 'stressed']
45 unique lemmatized words ["'m", 'a', 'am', 'angry', 'anxiety', 'anxious', 'bye', 'care', 'down', 'evening', 'feel', 'feeling', 'good', 'goodbye', 'great', 'happy', 'have', 'hello', 'hey', 'hi', 'i', 'in', 'lost', 'low', 'mad', 'mind', 'mood', 'morning', 'much', 'my', 'nervous', 'night', 'not', 'on', 'overwhelmed', 'positive', 'sad', 'see', 'stressed', 'take', 'temper', 'too', 'upset', 'worried', 'you']


In [5]:
pickle.dump(words, open("models/words.pkl", "wb"))
pickle.dump(classes, open("models/classes.pkl", "wb"))

In [6]:
# Training Data Preparation
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # Initialize bag of words
    bag = []
    # Lemmatize each word
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    # Create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # Fills `bag` with `1` if the word appears in the document, else `0`.


In [7]:
output_row = list(output_empty)
output_row[classes.index(doc[1])] = 1

training.append([bag, output_row])

In [8]:
random.shuffle(training)
training = np.array(training, dtype=object)

In [9]:
# Create train and test lists
train_x = np.array(list(training[:, 0]))
test_y = np.array(list(training[:, 1]))

print("Training data created")

Training data created


In [10]:
# Neural Network Model
model = Sequential([
    Dense(128, input_shape=(len(train_x[0]),), activation="relu"),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(len(test_y[0]), activation="softmax")
])


c:\Users\harsh\.conda\envs\Projects\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

c:\Users\harsh\.conda\envs\Projects\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [12]:
# Fit and save the model
hist = model.fit(train_x, test_y, epochs=200, batch_size=5, verbose=1)


Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step - accuracy: 1.0000 - loss: 1.7250
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 1.0000 - loss: 1.6054
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.0000e+00 - loss: 1.6968
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.0000e+00 - loss: 1.7436
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 1.0000 - loss: 0.9263
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 1.3174
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 1.0000 - loss: 0.5773
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 1.0000 - loss: 1.1076
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 1.0000 - loss: 0.4951
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 1.0000 - loss: 0.1856
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 1.0000 - loss: 0.0306
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 1.

In [13]:
model.save('models/emotion_model.h5', hist)
print("Model created")

Model created
